In [27]:
import os
import numpy
import matplotlib.pyplot as plt
import onnx
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer
import onnxruntime as rt
import sklearn
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import xgboost
from xgboost import XGBClassifier
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  # noqa
import onnxmltools.convert.common.data_types


In [28]:
dataPath = '/home/qinjianbo/DATA/guqinData/all.npy'
trainPercent = 0.8;

In [29]:
allData = numpy.load(dataPath)
numpy.random.shuffle(allData)

In [30]:
trainData = allData[0:int(allData.shape[0] * trainPercent),:]
testData = allData[int(allData.shape[0] * trainPercent):-1,:]

In [6]:
trainData.shape

(1388900, 40)

In [7]:
testData.shape

(347224, 40)

In [8]:
trainX = trainData[:,0:39]
trainY = trainData[:,-1]

In [75]:
xgboost = XGBClassifier(base_score=0.5,n_estimators=100,max_depth=6,learning_rate=0.01,max_leaves=100,objective='binary:logistic',eval_metric='logloss')

In [76]:
xgboost.fit(trainX,trainY)

XGBClassifier(base_score=0.5, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=100, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [64]:
initial_type = [('float_input', FloatTensorType([None, 39]))]

In [73]:
xgboost.save_model('./model')

In [66]:
from onnxmltools import convert_xgboost
from onnxmltools.utils import save_model
from onnxmltools.convert.common.data_types import FloatTensorType

In [74]:
initial_type = [('float_input', FloatTensorType([1, 39]))]
onx = convert_xgboost(xgboost, initial_types=initial_type)
save_model(onx, "xgboost.onnx")

In [70]:
xgboost.base_score

In [49]:
testData[0,:39].shape

(39,)

In [51]:
trainX[0]

array([-1.16356750e+03,  4.06575470e+01,  1.14428711e+00,  7.09395599e+00,
        4.20454025e+00,  2.51520920e+00,  1.83322906e+00, -4.52651978e-01,
        2.59569931e+00, -6.02062607e+00, -4.93874359e+00, -8.09817123e+00,
       -7.45995712e+00, -9.99784470e+00, -1.93315506e+00, -3.61482620e+00,
       -5.19104004e+00, -9.38163757e-01, -3.80905724e+00,  3.06747437e-01,
        2.45087433e+00, -2.63184547e+00, -4.38160133e+00, -2.83338737e+00,
        6.96800232e-01,  6.48811340e-01, -2.20756340e+00,  4.25691795e+00,
        3.51045036e+00,  1.12342834e-01, -3.28080750e+00, -1.56516647e+00,
        1.58039093e+00,  8.53785515e-01, -2.70013618e+00, -7.48783112e-01,
        5.78813553e-02,  1.11423492e+00, -1.59188747e+00])